In [2]:
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import dateutil.parser as dp
from datetime import datetime
from datetime import timedelta  
import time
from datetime import timezone
session = Session()

#converts pyton datatime to unix milisecond timestamp utc timezone
def to_utcunix(x):
    return int(x.replace(tzinfo=timezone.utc).timestamp()) * 1000

#Generator for binance price data
def get_data(start, batch_size, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_start = start
    
    while True:
        batch_end = batch_start + timedelta(minutes = batch_size)
        
        if(batch_end > end):
            batch_end = end - timedelta(minutes = 1)
        
            if(batch_start >= end- timedelta(minutes = 1)):
                break
        
        base = "https://api.binance.com/api/v3"

        print(batch_start, batch_end)
        
        #symbol = "BTCUSDT"
        #interval = "1m"
        
        url = base + "/klines?symbol={}&interval={}&startTime={}&endTime={}&limit={}".format(symbol, interval,
                                                                                     to_utcunix(batch_start)
                                                                                    , to_utcunix(batch_end), batch_size)
        
        data = session.get(url).json()
        batch_start = batch_end
        
        #sleep to stay under binance maximum call rate
        time.sleep(0.101)
        
        yield pd.DataFrame.from_dict(data)

#save binance price data as csv
def gen_csv(start, filename, end = datetime.now(), symbol = "BTCUSDT", interval = "1m"):
    batch_size = 1000
    d = get_data(start, batch_size, end = end, symbol = symbol, interval = interval)
    for i in d:
        i.to_csv("data_" + interval + "/" + filename + ".csv", mode = 'a', header = False)

#simplified gen_csv function
def gen_csv_year(year, symbol, interval = "1m"):
    gen_csv(datetime(year,1,1) , "{}_{}".format(year, symbol) , end = datetime(year + 1,1,1), interval = interval)
    

#for i in range(2016, 2021):
#    gen_csv_year(i, "ETHUSDT")    


In [3]:
for i in range(2017, 2020):

    gen_csv_year(i, "BTCUSDT", interval = "1h")

2017-01-01 00:00:00 2017-01-01 16:40:00
2017-01-01 16:40:00 2017-01-02 09:20:00
2017-01-02 09:20:00 2017-01-03 02:00:00
2017-01-03 02:00:00 2017-01-03 18:40:00
2017-01-03 18:40:00 2017-01-04 11:20:00
2017-01-04 11:20:00 2017-01-05 04:00:00
2017-01-05 04:00:00 2017-01-05 20:40:00
2017-01-05 20:40:00 2017-01-06 13:20:00
2017-01-06 13:20:00 2017-01-07 06:00:00
2017-01-07 06:00:00 2017-01-07 22:40:00
2017-01-07 22:40:00 2017-01-08 15:20:00
2017-01-08 15:20:00 2017-01-09 08:00:00
2017-01-09 08:00:00 2017-01-10 00:40:00
2017-01-10 00:40:00 2017-01-10 17:20:00
2017-01-10 17:20:00 2017-01-11 10:00:00
2017-01-11 10:00:00 2017-01-12 02:40:00
2017-01-12 02:40:00 2017-01-12 19:20:00
2017-01-12 19:20:00 2017-01-13 12:00:00
2017-01-13 12:00:00 2017-01-14 04:40:00
2017-01-14 04:40:00 2017-01-14 21:20:00
2017-01-14 21:20:00 2017-01-15 14:00:00
2017-01-15 14:00:00 2017-01-16 06:40:00
2017-01-16 06:40:00 2017-01-16 23:20:00
2017-01-16 23:20:00 2017-01-17 16:00:00
2017-01-17 16:00:00 2017-01-18 08:40:00


2017-05-23 08:40:00 2017-05-24 01:20:00
2017-05-24 01:20:00 2017-05-24 18:00:00
2017-05-24 18:00:00 2017-05-25 10:40:00
2017-05-25 10:40:00 2017-05-26 03:20:00
2017-05-26 03:20:00 2017-05-26 20:00:00
2017-05-26 20:00:00 2017-05-27 12:40:00
2017-05-27 12:40:00 2017-05-28 05:20:00
2017-05-28 05:20:00 2017-05-28 22:00:00
2017-05-28 22:00:00 2017-05-29 14:40:00
2017-05-29 14:40:00 2017-05-30 07:20:00
2017-05-30 07:20:00 2017-05-31 00:00:00
2017-05-31 00:00:00 2017-05-31 16:40:00
2017-05-31 16:40:00 2017-06-01 09:20:00
2017-06-01 09:20:00 2017-06-02 02:00:00
2017-06-02 02:00:00 2017-06-02 18:40:00
2017-06-02 18:40:00 2017-06-03 11:20:00
2017-06-03 11:20:00 2017-06-04 04:00:00
2017-06-04 04:00:00 2017-06-04 20:40:00
2017-06-04 20:40:00 2017-06-05 13:20:00
2017-06-05 13:20:00 2017-06-06 06:00:00
2017-06-06 06:00:00 2017-06-06 22:40:00
2017-06-06 22:40:00 2017-06-07 15:20:00
2017-06-07 15:20:00 2017-06-08 08:00:00
2017-06-08 08:00:00 2017-06-09 00:40:00
2017-06-09 00:40:00 2017-06-09 17:20:00


2017-10-12 17:20:00 2017-10-13 10:00:00
2017-10-13 10:00:00 2017-10-14 02:40:00
2017-10-14 02:40:00 2017-10-14 19:20:00
2017-10-14 19:20:00 2017-10-15 12:00:00
2017-10-15 12:00:00 2017-10-16 04:40:00
2017-10-16 04:40:00 2017-10-16 21:20:00
2017-10-16 21:20:00 2017-10-17 14:00:00
2017-10-17 14:00:00 2017-10-18 06:40:00
2017-10-18 06:40:00 2017-10-18 23:20:00
2017-10-18 23:20:00 2017-10-19 16:00:00
2017-10-19 16:00:00 2017-10-20 08:40:00
2017-10-20 08:40:00 2017-10-21 01:20:00
2017-10-21 01:20:00 2017-10-21 18:00:00
2017-10-21 18:00:00 2017-10-22 10:40:00
2017-10-22 10:40:00 2017-10-23 03:20:00
2017-10-23 03:20:00 2017-10-23 20:00:00
2017-10-23 20:00:00 2017-10-24 12:40:00
2017-10-24 12:40:00 2017-10-25 05:20:00
2017-10-25 05:20:00 2017-10-25 22:00:00
2017-10-25 22:00:00 2017-10-26 14:40:00
2017-10-26 14:40:00 2017-10-27 07:20:00
2017-10-27 07:20:00 2017-10-28 00:00:00
2017-10-28 00:00:00 2017-10-28 16:40:00
2017-10-28 16:40:00 2017-10-29 09:20:00
2017-10-29 09:20:00 2017-10-30 02:00:00


2018-03-03 19:20:00 2018-03-04 12:00:00
2018-03-04 12:00:00 2018-03-05 04:40:00
2018-03-05 04:40:00 2018-03-05 21:20:00
2018-03-05 21:20:00 2018-03-06 14:00:00
2018-03-06 14:00:00 2018-03-07 06:40:00
2018-03-07 06:40:00 2018-03-07 23:20:00
2018-03-07 23:20:00 2018-03-08 16:00:00
2018-03-08 16:00:00 2018-03-09 08:40:00
2018-03-09 08:40:00 2018-03-10 01:20:00
2018-03-10 01:20:00 2018-03-10 18:00:00
2018-03-10 18:00:00 2018-03-11 10:40:00
2018-03-11 10:40:00 2018-03-12 03:20:00
2018-03-12 03:20:00 2018-03-12 20:00:00
2018-03-12 20:00:00 2018-03-13 12:40:00
2018-03-13 12:40:00 2018-03-14 05:20:00
2018-03-14 05:20:00 2018-03-14 22:00:00
2018-03-14 22:00:00 2018-03-15 14:40:00
2018-03-15 14:40:00 2018-03-16 07:20:00
2018-03-16 07:20:00 2018-03-17 00:00:00
2018-03-17 00:00:00 2018-03-17 16:40:00
2018-03-17 16:40:00 2018-03-18 09:20:00
2018-03-18 09:20:00 2018-03-19 02:00:00
2018-03-19 02:00:00 2018-03-19 18:40:00
2018-03-19 18:40:00 2018-03-20 11:20:00
2018-03-20 11:20:00 2018-03-21 04:00:00


2018-07-24 04:00:00 2018-07-24 20:40:00
2018-07-24 20:40:00 2018-07-25 13:20:00
2018-07-25 13:20:00 2018-07-26 06:00:00
2018-07-26 06:00:00 2018-07-26 22:40:00
2018-07-26 22:40:00 2018-07-27 15:20:00
2018-07-27 15:20:00 2018-07-28 08:00:00
2018-07-28 08:00:00 2018-07-29 00:40:00
2018-07-29 00:40:00 2018-07-29 17:20:00
2018-07-29 17:20:00 2018-07-30 10:00:00
2018-07-30 10:00:00 2018-07-31 02:40:00
2018-07-31 02:40:00 2018-07-31 19:20:00
2018-07-31 19:20:00 2018-08-01 12:00:00
2018-08-01 12:00:00 2018-08-02 04:40:00
2018-08-02 04:40:00 2018-08-02 21:20:00
2018-08-02 21:20:00 2018-08-03 14:00:00
2018-08-03 14:00:00 2018-08-04 06:40:00
2018-08-04 06:40:00 2018-08-04 23:20:00
2018-08-04 23:20:00 2018-08-05 16:00:00
2018-08-05 16:00:00 2018-08-06 08:40:00
2018-08-06 08:40:00 2018-08-07 01:20:00
2018-08-07 01:20:00 2018-08-07 18:00:00
2018-08-07 18:00:00 2018-08-08 10:40:00
2018-08-08 10:40:00 2018-08-09 03:20:00
2018-08-09 03:20:00 2018-08-09 20:00:00
2018-08-09 20:00:00 2018-08-10 12:40:00


2018-12-13 12:40:00 2018-12-14 05:20:00
2018-12-14 05:20:00 2018-12-14 22:00:00
2018-12-14 22:00:00 2018-12-15 14:40:00
2018-12-15 14:40:00 2018-12-16 07:20:00
2018-12-16 07:20:00 2018-12-17 00:00:00
2018-12-17 00:00:00 2018-12-17 16:40:00
2018-12-17 16:40:00 2018-12-18 09:20:00
2018-12-18 09:20:00 2018-12-19 02:00:00
2018-12-19 02:00:00 2018-12-19 18:40:00
2018-12-19 18:40:00 2018-12-20 11:20:00
2018-12-20 11:20:00 2018-12-21 04:00:00
2018-12-21 04:00:00 2018-12-21 20:40:00
2018-12-21 20:40:00 2018-12-22 13:20:00
2018-12-22 13:20:00 2018-12-23 06:00:00
2018-12-23 06:00:00 2018-12-23 22:40:00
2018-12-23 22:40:00 2018-12-24 15:20:00
2018-12-24 15:20:00 2018-12-25 08:00:00
2018-12-25 08:00:00 2018-12-26 00:40:00
2018-12-26 00:40:00 2018-12-26 17:20:00
2018-12-26 17:20:00 2018-12-27 10:00:00
2018-12-27 10:00:00 2018-12-28 02:40:00
2018-12-28 02:40:00 2018-12-28 19:20:00
2018-12-28 19:20:00 2018-12-29 12:00:00
2018-12-29 12:00:00 2018-12-30 04:40:00
2018-12-30 04:40:00 2018-12-30 21:20:00


2019-05-04 14:40:00 2019-05-05 07:20:00
2019-05-05 07:20:00 2019-05-06 00:00:00
2019-05-06 00:00:00 2019-05-06 16:40:00
2019-05-06 16:40:00 2019-05-07 09:20:00
2019-05-07 09:20:00 2019-05-08 02:00:00
2019-05-08 02:00:00 2019-05-08 18:40:00
2019-05-08 18:40:00 2019-05-09 11:20:00
2019-05-09 11:20:00 2019-05-10 04:00:00
2019-05-10 04:00:00 2019-05-10 20:40:00
2019-05-10 20:40:00 2019-05-11 13:20:00
2019-05-11 13:20:00 2019-05-12 06:00:00
2019-05-12 06:00:00 2019-05-12 22:40:00
2019-05-12 22:40:00 2019-05-13 15:20:00
2019-05-13 15:20:00 2019-05-14 08:00:00
2019-05-14 08:00:00 2019-05-15 00:40:00
2019-05-15 00:40:00 2019-05-15 17:20:00
2019-05-15 17:20:00 2019-05-16 10:00:00
2019-05-16 10:00:00 2019-05-17 02:40:00
2019-05-17 02:40:00 2019-05-17 19:20:00
2019-05-17 19:20:00 2019-05-18 12:00:00
2019-05-18 12:00:00 2019-05-19 04:40:00
2019-05-19 04:40:00 2019-05-19 21:20:00
2019-05-19 21:20:00 2019-05-20 14:00:00
2019-05-20 14:00:00 2019-05-21 06:40:00
2019-05-21 06:40:00 2019-05-21 23:20:00


2019-09-23 23:20:00 2019-09-24 16:00:00
2019-09-24 16:00:00 2019-09-25 08:40:00
2019-09-25 08:40:00 2019-09-26 01:20:00
2019-09-26 01:20:00 2019-09-26 18:00:00
2019-09-26 18:00:00 2019-09-27 10:40:00
2019-09-27 10:40:00 2019-09-28 03:20:00
2019-09-28 03:20:00 2019-09-28 20:00:00
2019-09-28 20:00:00 2019-09-29 12:40:00
2019-09-29 12:40:00 2019-09-30 05:20:00
2019-09-30 05:20:00 2019-09-30 22:00:00
2019-09-30 22:00:00 2019-10-01 14:40:00
2019-10-01 14:40:00 2019-10-02 07:20:00
2019-10-02 07:20:00 2019-10-03 00:00:00
2019-10-03 00:00:00 2019-10-03 16:40:00
2019-10-03 16:40:00 2019-10-04 09:20:00
2019-10-04 09:20:00 2019-10-05 02:00:00
2019-10-05 02:00:00 2019-10-05 18:40:00
2019-10-05 18:40:00 2019-10-06 11:20:00
2019-10-06 11:20:00 2019-10-07 04:00:00
2019-10-07 04:00:00 2019-10-07 20:40:00
2019-10-07 20:40:00 2019-10-08 13:20:00
2019-10-08 13:20:00 2019-10-09 06:00:00
2019-10-09 06:00:00 2019-10-09 22:40:00
2019-10-09 22:40:00 2019-10-10 15:20:00
2019-10-10 15:20:00 2019-10-11 08:00:00


In [4]:
for i in range(2017, 2021):

    gen_csv_year(i, "ETHUSDT", interval = "5m")

2017-01-01 00:00:00 2017-01-01 16:40:00
2017-01-01 16:40:00 2017-01-02 09:20:00
2017-01-02 09:20:00 2017-01-03 02:00:00
2017-01-03 02:00:00 2017-01-03 18:40:00
2017-01-03 18:40:00 2017-01-04 11:20:00
2017-01-04 11:20:00 2017-01-05 04:00:00
2017-01-05 04:00:00 2017-01-05 20:40:00
2017-01-05 20:40:00 2017-01-06 13:20:00
2017-01-06 13:20:00 2017-01-07 06:00:00
2017-01-07 06:00:00 2017-01-07 22:40:00
2017-01-07 22:40:00 2017-01-08 15:20:00
2017-01-08 15:20:00 2017-01-09 08:00:00
2017-01-09 08:00:00 2017-01-10 00:40:00
2017-01-10 00:40:00 2017-01-10 17:20:00
2017-01-10 17:20:00 2017-01-11 10:00:00
2017-01-11 10:00:00 2017-01-12 02:40:00
2017-01-12 02:40:00 2017-01-12 19:20:00
2017-01-12 19:20:00 2017-01-13 12:00:00
2017-01-13 12:00:00 2017-01-14 04:40:00
2017-01-14 04:40:00 2017-01-14 21:20:00
2017-01-14 21:20:00 2017-01-15 14:00:00
2017-01-15 14:00:00 2017-01-16 06:40:00
2017-01-16 06:40:00 2017-01-16 23:20:00
2017-01-16 23:20:00 2017-01-17 16:00:00
2017-01-17 16:00:00 2017-01-18 08:40:00


2017-05-23 08:40:00 2017-05-24 01:20:00
2017-05-24 01:20:00 2017-05-24 18:00:00
2017-05-24 18:00:00 2017-05-25 10:40:00
2017-05-25 10:40:00 2017-05-26 03:20:00
2017-05-26 03:20:00 2017-05-26 20:00:00
2017-05-26 20:00:00 2017-05-27 12:40:00
2017-05-27 12:40:00 2017-05-28 05:20:00
2017-05-28 05:20:00 2017-05-28 22:00:00
2017-05-28 22:00:00 2017-05-29 14:40:00
2017-05-29 14:40:00 2017-05-30 07:20:00
2017-05-30 07:20:00 2017-05-31 00:00:00
2017-05-31 00:00:00 2017-05-31 16:40:00
2017-05-31 16:40:00 2017-06-01 09:20:00
2017-06-01 09:20:00 2017-06-02 02:00:00
2017-06-02 02:00:00 2017-06-02 18:40:00
2017-06-02 18:40:00 2017-06-03 11:20:00
2017-06-03 11:20:00 2017-06-04 04:00:00
2017-06-04 04:00:00 2017-06-04 20:40:00
2017-06-04 20:40:00 2017-06-05 13:20:00
2017-06-05 13:20:00 2017-06-06 06:00:00
2017-06-06 06:00:00 2017-06-06 22:40:00
2017-06-06 22:40:00 2017-06-07 15:20:00
2017-06-07 15:20:00 2017-06-08 08:00:00
2017-06-08 08:00:00 2017-06-09 00:40:00
2017-06-09 00:40:00 2017-06-09 17:20:00


2017-10-12 17:20:00 2017-10-13 10:00:00
2017-10-13 10:00:00 2017-10-14 02:40:00
2017-10-14 02:40:00 2017-10-14 19:20:00
2017-10-14 19:20:00 2017-10-15 12:00:00
2017-10-15 12:00:00 2017-10-16 04:40:00
2017-10-16 04:40:00 2017-10-16 21:20:00
2017-10-16 21:20:00 2017-10-17 14:00:00
2017-10-17 14:00:00 2017-10-18 06:40:00
2017-10-18 06:40:00 2017-10-18 23:20:00
2017-10-18 23:20:00 2017-10-19 16:00:00
2017-10-19 16:00:00 2017-10-20 08:40:00
2017-10-20 08:40:00 2017-10-21 01:20:00
2017-10-21 01:20:00 2017-10-21 18:00:00
2017-10-21 18:00:00 2017-10-22 10:40:00
2017-10-22 10:40:00 2017-10-23 03:20:00
2017-10-23 03:20:00 2017-10-23 20:00:00
2017-10-23 20:00:00 2017-10-24 12:40:00
2017-10-24 12:40:00 2017-10-25 05:20:00
2017-10-25 05:20:00 2017-10-25 22:00:00
2017-10-25 22:00:00 2017-10-26 14:40:00
2017-10-26 14:40:00 2017-10-27 07:20:00
2017-10-27 07:20:00 2017-10-28 00:00:00
2017-10-28 00:00:00 2017-10-28 16:40:00
2017-10-28 16:40:00 2017-10-29 09:20:00
2017-10-29 09:20:00 2017-10-30 02:00:00


2018-03-03 19:20:00 2018-03-04 12:00:00
2018-03-04 12:00:00 2018-03-05 04:40:00
2018-03-05 04:40:00 2018-03-05 21:20:00
2018-03-05 21:20:00 2018-03-06 14:00:00
2018-03-06 14:00:00 2018-03-07 06:40:00
2018-03-07 06:40:00 2018-03-07 23:20:00
2018-03-07 23:20:00 2018-03-08 16:00:00
2018-03-08 16:00:00 2018-03-09 08:40:00
2018-03-09 08:40:00 2018-03-10 01:20:00
2018-03-10 01:20:00 2018-03-10 18:00:00
2018-03-10 18:00:00 2018-03-11 10:40:00
2018-03-11 10:40:00 2018-03-12 03:20:00
2018-03-12 03:20:00 2018-03-12 20:00:00
2018-03-12 20:00:00 2018-03-13 12:40:00
2018-03-13 12:40:00 2018-03-14 05:20:00
2018-03-14 05:20:00 2018-03-14 22:00:00
2018-03-14 22:00:00 2018-03-15 14:40:00
2018-03-15 14:40:00 2018-03-16 07:20:00
2018-03-16 07:20:00 2018-03-17 00:00:00
2018-03-17 00:00:00 2018-03-17 16:40:00
2018-03-17 16:40:00 2018-03-18 09:20:00
2018-03-18 09:20:00 2018-03-19 02:00:00
2018-03-19 02:00:00 2018-03-19 18:40:00
2018-03-19 18:40:00 2018-03-20 11:20:00
2018-03-20 11:20:00 2018-03-21 04:00:00


2018-07-24 04:00:00 2018-07-24 20:40:00
2018-07-24 20:40:00 2018-07-25 13:20:00
2018-07-25 13:20:00 2018-07-26 06:00:00
2018-07-26 06:00:00 2018-07-26 22:40:00
2018-07-26 22:40:00 2018-07-27 15:20:00
2018-07-27 15:20:00 2018-07-28 08:00:00
2018-07-28 08:00:00 2018-07-29 00:40:00
2018-07-29 00:40:00 2018-07-29 17:20:00
2018-07-29 17:20:00 2018-07-30 10:00:00
2018-07-30 10:00:00 2018-07-31 02:40:00
2018-07-31 02:40:00 2018-07-31 19:20:00
2018-07-31 19:20:00 2018-08-01 12:00:00
2018-08-01 12:00:00 2018-08-02 04:40:00
2018-08-02 04:40:00 2018-08-02 21:20:00
2018-08-02 21:20:00 2018-08-03 14:00:00
2018-08-03 14:00:00 2018-08-04 06:40:00
2018-08-04 06:40:00 2018-08-04 23:20:00
2018-08-04 23:20:00 2018-08-05 16:00:00
2018-08-05 16:00:00 2018-08-06 08:40:00
2018-08-06 08:40:00 2018-08-07 01:20:00
2018-08-07 01:20:00 2018-08-07 18:00:00
2018-08-07 18:00:00 2018-08-08 10:40:00
2018-08-08 10:40:00 2018-08-09 03:20:00
2018-08-09 03:20:00 2018-08-09 20:00:00
2018-08-09 20:00:00 2018-08-10 12:40:00


...

2019-05-04 14:40:00 2019-05-05 07:20:00
2019-05-05 07:20:00 2019-05-06 00:00:00
2019-05-06 00:00:00 2019-05-06 16:40:00
2019-05-06 16:40:00 2019-05-07 09:20:00
2019-05-07 09:20:00 2019-05-08 02:00:00
2019-05-08 02:00:00 2019-05-08 18:40:00
2019-05-08 18:40:00 2019-05-09 11:20:00
2019-05-09 11:20:00 2019-05-10 04:00:00
2019-05-10 04:00:00 2019-05-10 20:40:00
2019-05-10 20:40:00 2019-05-11 13:20:00
2019-05-11 13:20:00 2019-05-12 06:00:00
2019-05-12 06:00:00 2019-05-12 22:40:00
2019-05-12 22:40:00 2019-05-13 15:20:00
2019-05-13 15:20:00 2019-05-14 08:00:00
2019-05-14 08:00:00 2019-05-15 00:40:00
2019-05-15 00:40:00 2019-05-15 17:20:00
2019-05-15 17:20:00 2019-05-16 10:00:00
2019-05-16 10:00:00 2019-05-17 02:40:00
2019-05-17 02:40:00 2019-05-17 19:20:00
2019-05-17 19:20:00 2019-05-18 12:00:00
2019-05-18 12:00:00 2019-05-19 04:40:00
2019-05-19 04:40:00 2019-05-19 21:20:00
2019-05-19 21:20:00 2019-05-20 14:00:00
2019-05-20 14:00:00 2019-05-21 06:40:00
2019-05-21 06:40:00 2019-05-21 23:20:00


2019-09-23 23:20:00 2019-09-24 16:00:00
2019-09-24 16:00:00 2019-09-25 08:40:00
2019-09-25 08:40:00 2019-09-26 01:20:00
2019-09-26 01:20:00 2019-09-26 18:00:00
2019-09-26 18:00:00 2019-09-27 10:40:00
2019-09-27 10:40:00 2019-09-28 03:20:00
2019-09-28 03:20:00 2019-09-28 20:00:00
2019-09-28 20:00:00 2019-09-29 12:40:00
2019-09-29 12:40:00 2019-09-30 05:20:00
2019-09-30 05:20:00 2019-09-30 22:00:00
2019-09-30 22:00:00 2019-10-01 14:40:00
2019-10-01 14:40:00 2019-10-02 07:20:00
2019-10-02 07:20:00 2019-10-03 00:00:00
2019-10-03 00:00:00 2019-10-03 16:40:00
2019-10-03 16:40:00 2019-10-04 09:20:00
2019-10-04 09:20:00 2019-10-05 02:00:00
2019-10-05 02:00:00 2019-10-05 18:40:00
2019-10-05 18:40:00 2019-10-06 11:20:00
2019-10-06 11:20:00 2019-10-07 04:00:00
2019-10-07 04:00:00 2019-10-07 20:40:00
2019-10-07 20:40:00 2019-10-08 13:20:00
2019-10-08 13:20:00 2019-10-09 06:00:00
2019-10-09 06:00:00 2019-10-09 22:40:00
2019-10-09 22:40:00 2019-10-10 15:20:00
2019-10-10 15:20:00 2019-10-11 08:00:00


2020-02-13 01:20:00 2020-02-13 18:00:00
2020-02-13 18:00:00 2020-02-14 10:40:00
2020-02-14 10:40:00 2020-02-15 03:20:00
2020-02-15 03:20:00 2020-02-15 20:00:00
2020-02-15 20:00:00 2020-02-16 12:40:00
2020-02-16 12:40:00 2020-02-17 05:20:00
2020-02-17 05:20:00 2020-02-17 22:00:00
2020-02-17 22:00:00 2020-02-18 14:40:00
2020-02-18 14:40:00 2020-02-19 07:20:00
2020-02-19 07:20:00 2020-02-20 00:00:00
2020-02-20 00:00:00 2020-02-20 16:40:00
2020-02-20 16:40:00 2020-02-21 09:20:00
2020-02-21 09:20:00 2020-02-22 02:00:00
2020-02-22 02:00:00 2020-02-22 18:40:00
2020-02-22 18:40:00 2020-02-23 11:20:00
2020-02-23 11:20:00 2020-02-24 04:00:00
2020-02-24 04:00:00 2020-02-24 20:40:00
2020-02-24 20:40:00 2020-02-25 13:20:00
2020-02-25 13:20:00 2020-02-26 06:00:00
2020-02-26 06:00:00 2020-02-26 22:40:00
2020-02-26 22:40:00 2020-02-27 15:20:00
2020-02-27 15:20:00 2020-02-28 08:00:00
2020-02-28 08:00:00 2020-02-29 00:40:00
2020-02-29 00:40:00 2020-02-29 17:20:00
2020-02-29 17:20:00 2020-03-01 10:00:00


2020-07-04 10:00:00 2020-07-05 02:40:00
2020-07-05 02:40:00 2020-07-05 19:20:00
2020-07-05 19:20:00 2020-07-06 12:00:00
2020-07-06 12:00:00 2020-07-07 04:40:00
2020-07-07 04:40:00 2020-07-07 21:20:00
2020-07-07 21:20:00 2020-07-08 14:00:00
2020-07-08 14:00:00 2020-07-09 06:40:00
2020-07-09 06:40:00 2020-07-09 23:20:00
2020-07-09 23:20:00 2020-07-10 16:00:00
2020-07-10 16:00:00 2020-07-11 08:40:00
2020-07-11 08:40:00 2020-07-12 01:20:00
2020-07-12 01:20:00 2020-07-12 18:00:00
2020-07-12 18:00:00 2020-07-13 10:40:00
2020-07-13 10:40:00 2020-07-14 03:20:00
2020-07-14 03:20:00 2020-07-14 20:00:00
2020-07-14 20:00:00 2020-07-15 12:40:00
2020-07-15 12:40:00 2020-07-16 05:20:00
2020-07-16 05:20:00 2020-07-16 22:00:00
2020-07-16 22:00:00 2020-07-17 14:40:00
2020-07-17 14:40:00 2020-07-18 07:20:00
2020-07-18 07:20:00 2020-07-19 00:00:00
2020-07-19 00:00:00 2020-07-19 16:40:00
2020-07-19 16:40:00 2020-07-20 09:20:00
2020-07-20 09:20:00 2020-07-21 02:00:00
2020-07-21 02:00:00 2020-07-21 18:40:00


2020-11-23 18:40:00 2020-11-24 11:20:00
2020-11-24 11:20:00 2020-11-25 04:00:00
2020-11-25 04:00:00 2020-11-25 20:40:00
2020-11-25 20:40:00 2020-11-26 13:20:00
2020-11-26 13:20:00 2020-11-27 06:00:00
2020-11-27 06:00:00 2020-11-27 22:40:00
2020-11-27 22:40:00 2020-11-28 15:20:00
2020-11-28 15:20:00 2020-11-29 08:00:00
2020-11-29 08:00:00 2020-11-30 00:40:00
2020-11-30 00:40:00 2020-11-30 17:20:00
2020-11-30 17:20:00 2020-12-01 10:00:00
2020-12-01 10:00:00 2020-12-02 02:40:00
2020-12-02 02:40:00 2020-12-02 19:20:00
2020-12-02 19:20:00 2020-12-03 12:00:00
2020-12-03 12:00:00 2020-12-04 04:40:00
2020-12-04 04:40:00 2020-12-04 21:20:00
2020-12-04 21:20:00 2020-12-05 14:00:00
2020-12-05 14:00:00 2020-12-06 06:40:00
2020-12-06 06:40:00 2020-12-06 23:20:00
2020-12-06 23:20:00 2020-12-07 16:00:00
2020-12-07 16:00:00 2020-12-08 08:40:00
2020-12-08 08:40:00 2020-12-09 01:20:00
2020-12-09 01:20:00 2020-12-09 18:00:00
2020-12-09 18:00:00 2020-12-10 10:40:00
2020-12-10 10:40:00 2020-12-11 03:20:00
